In [6]:
#import packages
import pandas as pd
import numpy as np
import folium # map rendering library
from geopy.distance import geodesic #calculate distance between two points
from geopy.geocoders import Nominatim #convert address to latitude and longitude
import matplotlib.pyplot as plt # plotting
from scipy.signal import savgol_filter #for smoothing
from sklearn.cluster import DBSCAN #for clustering
from sklearn.neighbors import BallTree #for fast nearest neighbor search

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [42]:
# Step 1: Load the data
data = pd.read_csv('mynewdata.csv')

# Step 2: Sort the data by timestamp
data.sort_values('TimeStamp', inplace=True)

# Step 3: Create a map centered around the first coordinate
m = folium.Map(location=[data.iloc[0]['Latitude'], data.iloc[0]['Longitude']], zoom_start=20)

# Step 4: Plot the trajectory as a line on the map
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude']), 
                     (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath = 'mynewdata_walk_pattern_map.html'
m.save(map_filepath)

map_filepath


'mynewdata_walk_pattern_map.html'

In [43]:
# shape of the data
data.shape

(240, 24)

In [44]:
# Step 1: Handle missing values by removing rows with NaN values in Latitude and Longitude columns
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Step 2: Remove rows with same Latitude and Longitude as the previous row
#data = data.loc[(data['Latitude'].shift() != data['Latitude']) | (data['Longitude'].shift() != data['Longitude'])]

# Step 3: Remove random jumps by setting a threshold for the maximum allowable distance between consecutive points (e.g., 100 meters)
max_distance_m = 100
for i in range(1, len(data)):
    coord1 = (data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude'])
    coord2 = (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])
    if geodesic(coord1, coord2).meters > max_distance_m:
        data.iloc[i, data.columns.get_loc('Latitude')] = data.iloc[i-1]['Latitude']
        data.iloc[i, data.columns.get_loc('Longitude')] = data.iloc[i-1]['Longitude']

# Display the first few rows of the preprocessed data
data.head(100)


,Soldier_ID,TimeStamp,HeartRate,Latitude,Longitude,Accelerometer_X,Accelerometer_Y,Accelerometer_Z,Linear_Accelerometer_X,Linear_Accelerometer_Y,...,uncalib_Gyro_X,uncalib_Gyro_Y,uncalib_Gyro_Z,Gyro_rotation_vector_X,Gyro_rotation_vector_Y,Gyro_rotation_vector_Z,Gravity_X,Gravity_Y,Gravity_Z,User_Activity
0,0,10/6/23 3:54 PM,124,33.646182,72.998279,-6.83629,-1.71326,6.93440,-16.64629,-11.52326,...,-16.59000,-2.10,0.07,0.91297,0.26718,-0.00475,-7.16585,-1.70819,6.47326,Start_Activity
30,0,10/6/23 3:54 PM,127,33.646182,72.998279,-7.23829,2.21097,3.93619,-17.04829,-7.59903,...,-24.92000,-15.68,0.07,0.86311,-0.08887,0.21208,-8.35255,1.47956,4.92099,Start_Activity
31,0,10/6/23 3:54 PM,128,33.646182,72.998279,-15.51745,2.97906,5.07996,-25.32745,-6.83094,...,1.05000,47.81,0.07,0.83439,-0.11636,0.13275,-8.60266,1.77894,4.35889,Start_Activity
32,0,10/6/23 3:54 PM,128,33.646182,72.998279,-9.91586,1.54576,3.73998,-19.72586,-8.26424,...,-25.90000,8.54,0.07,0.82222,-0.09347,0.15839,-8.48067,1.83988,4.56765,Start_Activity
33,0,10/6/23 3:54 PM,129,33.646182,72.998279,-8.44906,0.82792,3.85244,-18.25906,-8.98208,...,-57.75000,-22.12,0.07,0.84961,-0.03408,0.09224,-7.68571,1.48746,5.90658,Start_Activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0,10/6/23 3:55 PM,131,33.646182,72.998279,-9.10948,0.35653,3.35952,-18.91948,-9.45347,...,-31.29000,-63.42,0.07,0.81816,-0.11571,-0.01905,-9.16172,2.96230,1.85959,Start_Activity
64,0,10/6/23 3:55 PM,132,33.646182,72.998279,-4.38604,-3.01256,2.67039,-14.19604,-12.82256,...,124.60000,-31.15,0.07,0.81968,0.27005,-0.29894,-8.52971,-3.51230,3.32838,Start_Activity
65,0,10/6/23 3:55 PM,133,33.646182,72.998279,-15.60599,2.51486,4.56311,-25.41599,-7.29514,...,-8.33000,24.36,0.07,0.75664,-0.26716,0.10160,-8.67223,2.65034,-3.73343,Start_Activity
66,0,10/6/23 3:55 PM,132,33.646182,72.998279,-15.71606,1.50987,2.88574,-25.52606,-8.30013,...,-53.20000,9.03,0.07,0.72234,-0.26375,0.08545,-9.71290,0.90587,-1.00464,Start_Activity


In [45]:
#shape of the data
data.shape

(240, 24)

In [46]:
# Create a map centered around the first coordinate of the preprocessed data
m = folium.Map(location=[data.iloc[0]['Latitude'], data.iloc[0]['Longitude']], zoom_start=20)

# Plot the trajectory as a line on the map using the preprocessed data
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude'], data.iloc[i-1]['Longitude']), 
                     (data.iloc[i]['Latitude'], data.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath = 'mynewdata_walk_pattern_map_preprocessed.html'
m.save(map_filepath)

map_filepath

'mynewdata_walk_pattern_map_preprocessed.html'

In [5]:
# Apply a moving average filter with a window size of 5 to the Latitude and Longitude columns to smooth the data
window_size = 5
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Create a map centered around the first coordinate of the smoothed data
m = folium.Map(location=[data.iloc[0]['Latitude_smooth'], data.iloc[0]['Longitude_smooth']], zoom_start=20)

# Plot the trajectory as a line on the map using the smoothed data
for i in range(1, len(data)):
    folium.PolyLine([(data.iloc[i-1]['Latitude_smooth'], data.iloc[i-1]['Longitude_smooth']), 
                     (data.iloc[i]['Latitude_smooth'], data.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_smooth = 'walk_pattern_map_smooth.html'
m.save(map_filepath_smooth)

map_filepath_smooth


'walk_pattern_map_smooth.html'

In [35]:
# Convert the TimeStamp column to datetime data type
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])

# Set the TimeStamp column as the index
data.set_index('TimeStamp', inplace=True)

# Resample the data to obtain a single location per minute by taking the mean of the Latitude and Longitude values within each minute
data_resampled = data[['Latitude', 'Longitude']].resample('1T').mean().dropna()

# Create a map centered around the first coordinate of the resampled data
m = folium.Map(location=[data_resampled.iloc[0]['Latitude'], data_resampled.iloc[0]['Longitude']], zoom_start=20)

# Plot the trajectory as a line on the map using the resampled data
for i in range(1, len(data_resampled)):
    folium.PolyLine([(data_resampled.iloc[i-1]['Latitude'], data_resampled.iloc[i-1]['Longitude']), 
                     (data_resampled.iloc[i]['Latitude'], data_resampled.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled = 'walk_pattern_map_resampled.html'
m.save(map_filepath_resampled)

map_filepath_resampled


'walk_pattern_map_resampled.html'

In [36]:
# Resample the data to obtain a single location every 2 minutes by taking the mean of the Latitude and Longitude values within each 2-minute interval
data_resampled_2min = data[['Latitude', 'Longitude']].resample('2T').mean().dropna()

# Create a map centered around the first coordinate of the 2-minute resampled data
m = folium.Map(location=[data_resampled_2min.iloc[0]['Latitude'], data_resampled_2min.iloc[0]['Longitude']], zoom_start=20)

# Plot the trajectory as a line on the map using the 2-minute resampled data
for i in range(1, len(data_resampled_2min)):
    folium.PolyLine([(data_resampled_2min.iloc[i-1]['Latitude'], data_resampled_2min.iloc[i-1]['Longitude']), 
                     (data_resampled_2min.iloc[i]['Latitude'], data_resampled_2min.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_2min = 'walk_pattern_map_resampled_2min.html'
m.save(map_filepath_resampled_2min)

map_filepath_resampled_2min


'walk_pattern_map_resampled_2min.html'

In [37]:
# Resample the data to obtain a single location every 5 minutes by taking the mean of the Latitude and Longitude values within each 5-minute interval
data_resampled_5min = data[['Latitude', 'Longitude']].resample('5T').mean().dropna()

# Create a map centered around the first coordinate of the 5-minute resampled data
m = folium.Map(location=[data_resampled_5min.iloc[0]['Latitude'], data_resampled_5min.iloc[0]['Longitude']], zoom_start=20)

# Plot the trajectory as a line on the map using the 5-minute resampled data
for i in range(1, len(data_resampled_5min)):
    folium.PolyLine([(data_resampled_5min.iloc[i-1]['Latitude'], data_resampled_5min.iloc[i-1]['Longitude']), 
                     (data_resampled_5min.iloc[i]['Latitude'], data_resampled_5min.iloc[i]['Longitude'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_5min = 'walk_pattern_map_resampled_5min.html'
m.save(map_filepath_resampled_5min)

map_filepath_resampled_5min


'walk_pattern_map_resampled_5min.html'

In [38]:
# Apply a moving average filter with a window size of 3 to the 2-minute resampled Latitude and Longitude columns to smooth the data
window_size = 3
data_resampled_2min['Latitude_smooth'] = data_resampled_2min['Latitude'].rolling(window=window_size, center=True).mean()
data_resampled_2min['Longitude_smooth'] = data_resampled_2min['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data_resampled_2min.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Create a map centered around the first coordinate of the smoothed and resampled data
m = folium.Map(location=[data_resampled_2min.iloc[0]['Latitude_smooth'], data_resampled_2min.iloc[0]['Longitude_smooth']], zoom_start=20)

# Plot the trajectory as a line on the map using the smoothed and resampled data
for i in range(1, len(data_resampled_2min)):
    folium.PolyLine([(data_resampled_2min.iloc[i-1]['Latitude_smooth'], data_resampled_2min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_2min.iloc[i]['Latitude_smooth'], data_resampled_2min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_smooth_2min = 'walk_pattern_map_resampled_smooth_2min.html'
m.save(map_filepath_resampled_smooth_2min)

map_filepath_resampled_smooth_2min


'walk_pattern_map_resampled_smooth_2min.html'

In [39]:
# Apply a moving average filter with a window size of 3 to the original Latitude and Longitude columns to smooth the data
window_size = 5
data['Latitude_smooth'] = data['Latitude'].rolling(window=window_size, center=True).mean()
data['Longitude_smooth'] = data['Longitude'].rolling(window=window_size, center=True).mean()

# Remove the rows with NaN values generated due to smoothing
data.dropna(subset=['Latitude_smooth', 'Longitude_smooth'], inplace=True)

# Resample the smoothed data to obtain a single location every 1 minute
data_resampled_smooth_1min = data[['Latitude_smooth', 'Longitude_smooth']].resample('1T').mean().dropna()

# Create a map centered around the first coordinate of the 1-minute resampled and smoothed data
m = folium.Map(location=[data_resampled_smooth_1min.iloc[0]['Latitude_smooth'], data_resampled_smooth_1min.iloc[0]['Longitude_smooth']], zoom_start=20)

# Plot the trajectory as a line on the map using the 1-minute resampled and smoothed data
for i in range(1, len(data_resampled_smooth_1min)):
    folium.PolyLine([(data_resampled_smooth_1min.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_smooth_1min.iloc[i]['Latitude_smooth'], data_resampled_smooth_1min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_resampled_smooth_1min = 'walk_pattern_map_resampled_smooth_1min.html'
m.save(map_filepath_resampled_smooth_1min)

map_filepath_resampled_smooth_1min


'walk_pattern_map_resampled_smooth_1min.html'

In [46]:
# Get the total length of the data and the time range it covers
data_length = len(data)
data_time_range = data.index[-1] - data.index[0]

data_length, data_time_range


(443, Timedelta('4 days 19:37:35.226000'))

In [42]:
from scipy.signal import savgol_filter

# Apply Savitzky-Golay smoothing with a window length of 5 and polynomial order of 2
data['Latitude_smooth'] = savgol_filter(data['Latitude'], window_length=5, polyorder=2)
data['Longitude_smooth'] = savgol_filter(data['Longitude'], window_length=5, polyorder=2)

# Resample the smoothed data to obtain a single location every 2 minutes
data_resampled_smooth_2min = data[['Latitude_smooth', 'Longitude_smooth']].resample('2T').mean().dropna()

# Create a map centered around the first coordinate of the 2-minute resampled and smoothed data
m = folium.Map(location=[data_resampled_smooth_2min.iloc[0]['Latitude_smooth'], data_resampled_smooth_2min.iloc[0]['Longitude_smooth']], zoom_start=20)

# Plot the trajectory as a line on the map using the 2-minute resampled and smoothed data
for i in range(1, len(data_resampled_smooth_2min)):
    folium.PolyLine([(data_resampled_smooth_2min.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_2min.iloc[i-1]['Longitude_smooth']), 
                     (data_resampled_smooth_2min.iloc[i]['Latitude_smooth'], data_resampled_smooth_2min.iloc[i]['Longitude_smooth'])], color='blue').add_to(m)
    
# Save the map to an HTML file
map_filepath_savgol_2min = 'walk_pattern_map_savgol_resampled_2min.html'
m.save(map_filepath_savgol_2min)

map_filepath_savgol_2min


'walk_pattern_map_savgol_resampled_2min.html'

In [44]:
# Apply Savitzky-Golay smoothing with a window length of 5 and polynomial order of 2
data['Latitude_smooth'] = savgol_filter(data['Latitude'], window_length=5, polyorder=2)
data['Longitude_smooth'] = savgol_filter(data['Longitude'], window_length=5, polyorder=2)

# Resample the smoothed data to obtain a single location every 30 seconds
data_resampled_smooth_30s = data[['Latitude_smooth', 'Longitude_smooth']].resample('30S').mean().dropna()

# Create a map centered around the first coordinate of the 30-second resampled and smoothed data
m = folium.Map(location=[data_resampled_smooth_30s.iloc[0]['Latitude_smooth'], data_resampled_smooth_30s.iloc[0]['Longitude_smooth']], zoom_start=16)

# Plot the trajectory as a line on the map using the 30-second resampled and smoothed data
# Using different colors for paths with a minimum of 30 seconds difference at the same place
previous_coord = None
for i in range(1, len(data_resampled_smooth_30s)):
    current_coord = (data_resampled_smooth_30s.iloc[i]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i]['Longitude_smooth'])
    color = 'blue' if current_coord != previous_coord else 'red'
    
    # Highlight the areas where the person has been stationary for at least 30 seconds
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    
    folium.PolyLine([(data_resampled_smooth_30s.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i-1]['Longitude_smooth']), 
                     current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map to an HTML file
map_filepath_savgol_30s_stationary = 'walk_pattern_map_savgol_resampled_30s_stationary.html'
m.save(map_filepath_savgol_30s_stationary)

map_filepath_savgol_30s_stationary


'walk_pattern_map_savgol_resampled_30s_stationary.html'

In [45]:
from sklearn.cluster import DBSCAN

# Step 1: Identify key points using DBSCAN clustering to find areas of high density (e.g., stops) and low density (e.g., significant changes in direction)
coords = data[['Latitude', 'Longitude']].values
clustering = DBSCAN(eps=0.0001, min_samples=3).fit(coords)
data['Cluster'] = clustering.labels_

# Step 2: Apply a less aggressive Savitzky-Golay smoothing while retaining the identified key points
window_length = 5
data['Latitude_smooth'] = savgol_filter(data['Latitude'], window_length, polyorder=2)
data['Longitude_smooth'] = savgol_filter(data['Longitude'], window_length, polyorder=2)

# Retain the original coordinates for the identified key points
key_points_mask = data['Cluster'] != -1
data.loc[key_points_mask, 'Latitude_smooth'] = data.loc[key_points_mask, 'Latitude']
data.loc[key_points_mask, 'Longitude_smooth'] = data.loc[key_points_mask, 'Longitude']

# Step 3: Resample the data at a 30-second interval
data_resampled_smooth_30s = data[['Latitude_smooth', 'Longitude_smooth']].resample('30S').mean().dropna()

# Step 4: Create a map and plot the smoothed, resampled path, highlighting areas where the person was stationary for at least 30 seconds
m = folium.Map(location=[data_resampled_smooth_30s.iloc[0]['Latitude_smooth'], data_resampled_smooth_30s.iloc[0]['Longitude_smooth']], zoom_start=16)

previous_coord = None
for i in range(1, len(data_resampled_smooth_30s)):
    current_coord = (data_resampled_smooth_30s.iloc[i]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i]['Longitude_smooth'])
    color = 'blue' if current_coord != previous_coord else 'red'
    
    # Highlight the areas where the person has been stationary for at least 30 seconds
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    
    folium.PolyLine([(data_resampled_smooth_30s.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i-1]['Longitude_smooth']), 
                     current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map to an HTML file
map_filepath_dbscan_savgol_30s = 'walk_pattern_map_dbscan_savgol_resampled_30s.html'
m.save(map_filepath_dbscan_savgol_30s)

map_filepath_dbscan_savgol_30s


'walk_pattern_map_dbscan_savgol_resampled_30s.html'

In [47]:
# Get the total length of the data and the time range it covers for the "Live Sol" soldier
live_sol_data = data[data['Soldier_ID'] == 'Live Sol']
live_sol_data_length = len(live_sol_data)
live_sol_data_time_range = live_sol_data.index[-1] - live_sol_data.index[0]

live_sol_data_length, live_sol_data_time_range


(287, Timedelta('0 days 00:38:49.530000'))

In [51]:
# Step 1: Identify key points using DBSCAN clustering to find areas of high density (e.g., stops) and low density (e.g., significant changes in direction)
coords = live_sol_data[['Latitude', 'Longitude']].values
clustering = DBSCAN(eps=0.0001, min_samples=3).fit(coords)
live_sol_data['Cluster'] = clustering.labels_

# Step 2: Apply a slightly more aggressive Savitzky-Golay smoothing while retaining the identified key points
window_length = 7
live_sol_data['Latitude_smooth'] = savgol_filter(live_sol_data['Latitude'], window_length, polyorder=2)
live_sol_data['Longitude_smooth'] = savgol_filter(live_sol_data['Longitude'], window_length, polyorder=2)

# Retain the original coordinates for the identified key points
key_points_mask = live_sol_data['Cluster'] != -1
live_sol_data.loc[key_points_mask, 'Latitude_smooth'] = live_sol_data.loc[key_points_mask, 'Latitude']
live_sol_data.loc[key_points_mask, 'Longitude_smooth'] = live_sol_data.loc[key_points_mask, 'Longitude']

# Step 3: Resample the data at a 30-second interval
data_resampled_smooth_30s = live_sol_data[['Latitude_smooth', 'Longitude_smooth']].resample('30S').mean().dropna()

# Step 4: Create a map and plot the smoothed, resampled path, highlighting areas where the person was stationary for at least 1 minute
m = folium.Map(location=[data_resampled_smooth_30s.iloc[0]['Latitude_smooth'], data_resampled_smooth_30s.iloc[0]['Longitude_smooth']], zoom_start=20)

previous_coord = None
for i in range(1, len(data_resampled_smooth_30s)):
    current_coord = (data_resampled_smooth_30s.iloc[i]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i]['Longitude_smooth'])
    color = 'blue' if current_coord != previous_coord else 'red'
    
    # Highlight the areas where the person has been stationary for at least 1 minute
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    
    folium.PolyLine([(data_resampled_smooth_30s.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i-1]['Longitude_smooth']), 
                     current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map to an HTML file
map_filepath_dbscan_savgol_30s_live_sol = 'walk_pattern_map_dbscan_savgol_resampled_30s_live_sol.html'
m.save(map_filepath_dbscan_savgol_30s_live_sol)

map_filepath_dbscan_savgol_30s_live_sol


'walk_pattern_map_dbscan_savgol_resampled_30s_live_sol.html'

In [53]:
from sklearn.neighbors import BallTree

# Step 1: Identify sections of the path that have been crossed again
# We will use a BallTree to efficiently find points that are close to each other in space
tree = BallTree(live_sol_data[['Latitude_smooth', 'Longitude_smooth']].values)

# Find points that are within 5 meters of each other and at least 2 minutes apart in time
radius = 0.000045  # Approximate radius for 5 meters
time_difference_threshold = pd.Timedelta(minutes=2)

crossed_path_indices = []
for i, (index, row) in enumerate(live_sol_data.iterrows()):
    # Find points that are spatially close
    indices = tree.query_radius([[row['Latitude_smooth'], row['Longitude_smooth']]], r=radius)[0]
    
    # Filter points that are also sufficiently apart in time to identify crossed paths
    time_differences = live_sol_data.iloc[indices].index - index
    crossed_indices = indices[np.abs(time_differences) > time_difference_threshold]
    crossed_path_indices.extend(crossed_indices.tolist())

# Step 2: Create a map and plot the smoothed, resampled path, highlighting crossed paths and stationary areas
m = folium.Map(location=[data_resampled_smooth_30s.iloc[0]['Latitude_smooth'], data_resampled_smooth_30s.iloc[0]['Longitude_smooth']], zoom_start=16)

previous_coord = None
for i in range(1, len(data_resampled_smooth_30s)):
    current_coord = (data_resampled_smooth_30s.iloc[i]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i]['Longitude_smooth'])
    
    # Determine the color for the path segment
    if i in crossed_path_indices:
        color = 'purple'
    elif current_coord == previous_coord:
        color = 'red'
    else:
        color = 'blue'
    
    # Highlight the areas where the person has been stationary for at least 1 minute
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    
    folium.PolyLine([(data_resampled_smooth_30s.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_30s.iloc[i-1]['Longitude_smooth']), 
                     current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map to an HTML file
map_filepath_crossed_paths = 'walk_pattern_map_crossed_paths.html'
m.save(map_filepath_crossed_paths)

map_filepath_crossed_paths


'walk_pattern_map_crossed_paths.html'

In [ ]:
# Step 3: Resample the data at a 15-second interval to maintain a detailed representation of the walk pattern
data_resampled_smooth_15s = live_sol_data[['Latitude_smooth', 'Longitude_smooth']].resample('15S').mean().dropna()

# Step 4: Create a map and plot the smoothed, resampled path, highlighting crossed paths and stationary areas
m = folium.Map(location=[data_resampled_smooth_15s.iloc[0]['Latitude_smooth'], data_resampled_smooth_15s.iloc[0]['Longitude_smooth']], zoom_start=16)

previous_coord = None
for i in range(1, len(data_resampled_smooth_15s)):
    current_coord = (data_resampled_smooth_15s.iloc[i]['Latitude_smooth'], data_resampled_smooth_15s.iloc[i]['Longitude_smooth'])
    
    # Determine the color for the path segment
    if i in crossed_path_indices:
        color = 'purple'
    elif current_coord == previous_coord:
        color = 'red'
    else:
        color = 'blue'
    
    # Highlight the areas where the person has been stationary for at least 1 minute
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    
    folium.PolyLine([(data_resampled_smooth_15s.iloc[i-1]['Latitude_smooth'], data_resampled_smooth_15s.iloc[i-1]['Longitude_smooth']), 
                     current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map to an HTML file
map_filepath_crossed_paths_15s = 'walk_pattern_map_crossed_paths_15s.html'
m.save(map_filepath_crossed_paths_15s)

map_filepath_crossed_paths_15s


In [61]:
# Load the data
live_sol_data = data[data['Soldier_ID'] == 'Live Sol']

# Define your parameters here
window_length = 11  # Smoothing window size (must be odd, increase to smooth more)
resampling_interval = '30S'  # Resampling interval (e.g., '10S' for 10 seconds)
time_difference_threshold_min = 1  # Time difference threshold for identifying crossed paths in minutes
radius = 0.000045  # Spatial radius for identifying crossed paths (approx. 5 meters in degrees)

# Identify key points
clustering = DBSCAN(eps=0.0001, min_samples=3).fit(live_sol_data[['Latitude', 'Longitude']].values)
live_sol_data['Cluster'] = clustering.labels_

# Smooth the data
live_sol_data['Latitude_smooth'] = savgol_filter(live_sol_data['Latitude'], window_length, 2)
live_sol_data['Longitude_smooth'] = savgol_filter(live_sol_data['Longitude'], window_length, 2)

# Retain original coordinates for key points
key_points_mask = live_sol_data['Cluster'] != -1
live_sol_data.loc[key_points_mask, 'Latitude_smooth'] = live_sol_data.loc[key_points_mask, 'Latitude']
live_sol_data.loc[key_points_mask, 'Longitude_smooth'] = live_sol_data.loc[key_points_mask, 'Longitude']

# Identify crossed paths
tree = BallTree(live_sol_data[['Latitude_smooth', 'Longitude_smooth']].values)
crossed_path_indices = []
for i, (index, row) in enumerate(live_sol_data.iterrows()):
    indices = tree.query_radius([[row['Latitude_smooth'], row['Longitude_smooth']]], r=radius)[0]
    time_differences = live_sol_data.iloc[indices].index - index
    time_difference_threshold = pd.Timedelta(minutes=time_difference_threshold_min)
    crossed_indices = indices[np.abs(time_differences) > time_difference_threshold]
    crossed_path_indices.extend(crossed_indices.tolist())

# Resample the data
data_resampled_smooth = live_sol_data[['Latitude_smooth', 'Longitude_smooth']].resample(resampling_interval).mean().dropna()

# Create the map
m = folium.Map(location=[data_resampled_smooth.iloc[0]['Latitude_smooth'], data_resampled_smooth.iloc[0]['Longitude_smooth']], zoom_start=16)
previous_coord = None
for i in range(1, len(data_resampled_smooth)):
    current_coord = (data_resampled_smooth.iloc[i]['Latitude_smooth'], data_resampled_smooth.iloc[i]['Longitude_smooth'])
    color = 'purple' if i in crossed_path_indices else 'red' if current_coord == previous_coord else 'blue'
    if color == 'red':
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)
    folium.PolyLine([(data_resampled_smooth.iloc[i-1]['Latitude_smooth'], data_resampled_smooth.iloc[i-1]['Longitude_smooth']), current_coord], color=color).add_to(m)
    previous_coord = current_coord

# Save the map
map_filepath = 'walk_pattern_map_crossed_paths_10s.html'
m.save(map_filepath)

#display the map
#m


In [65]:
# Load the data
live_sol_data = data[data['Soldier_ID'] == 'Live Sol']

# Define your parameters here
window_length = 5  # Smoothing window size (must be odd, increase to smooth more)
resampling_interval = '30S'  # Resampling interval (e.g., '10S' for 10 seconds)
time_difference_threshold_min = 1  # Time difference threshold for identifying crossed paths in minutes
radius = 0.000045  # Spatial radius for identifying crossed paths (approx. 5 meters in degrees)

# Identify key points
clustering = DBSCAN(eps=0.0001, min_samples=3).fit(live_sol_data[['Latitude', 'Longitude']].values)
live_sol_data['Cluster'] = clustering.labels_

# Smooth the data
live_sol_data['Latitude_smooth'] = savgol_filter(live_sol_data['Latitude'], window_length, 2)
live_sol_data['Longitude_smooth'] = savgol_filter(live_sol_data['Longitude'], window_length, 2)

# Retain original coordinates for key points
key_points_mask = live_sol_data['Cluster'] != -1
live_sol_data.loc[key_points_mask, 'Latitude_smooth'] = live_sol_data.loc[key_points_mask, 'Latitude']
live_sol_data.loc[key_points_mask, 'Longitude_smooth'] = live_sol_data.loc[key_points_mask, 'Longitude']

# Identify crossed paths
tree = BallTree(live_sol_data[['Latitude_smooth', 'Longitude_smooth']].values)
crossed_path_indices = []
for i, (index, row) in enumerate(live_sol_data.iterrows()):
    indices = tree.query_radius([[row['Latitude_smooth'], row['Longitude_smooth']]], r=radius)[0]
    time_differences = live_sol_data.iloc[indices].index - index
    time_difference_threshold = pd.Timedelta(minutes=time_difference_threshold_min)
    crossed_indices = indices[np.abs(time_differences) > time_difference_threshold]
    crossed_path_indices.extend(crossed_indices.tolist())

# Resample the data
data_resampled_smooth = live_sol_data[['Latitude_smooth', 'Longitude_smooth']].resample(resampling_interval).mean().dropna()

# Create the map
m = folium.Map(location=[data_resampled_smooth.iloc[0]['Latitude_smooth'], data_resampled_smooth.iloc[0]['Longitude_smooth']], zoom_start=16)

# Identify the segments of the path
path_segments = []
current_segment = [0]
for i in range(1, len(data_resampled_smooth)):
    if i in crossed_path_indices:
        path_segments.append(current_segment)
        current_segment = [i]
    else:
        current_segment.append(i)
path_segments.append(current_segment)

# Plot each segment with a different color
colors = ['blue', 'green', 'red', 'purple', 'orange', 'darkblue', 'darkgreen', 'darkred']
for segment_indices, color in zip(path_segments, colors * (len(path_segments) // len(colors) + 1)):
    segment_coords = [(data_resampled_smooth.iloc[i]['Latitude_smooth'], data_resampled_smooth.iloc[i]['Longitude_smooth']) for i in segment_indices]
    folium.PolyLine(segment_coords, color=color).add_to(m)

# Highlight the areas where the person has been stationary for at least 1 minute
for i in range(1, len(data_resampled_smooth)):
    current_coord = (data_resampled_smooth.iloc[i]['Latitude_smooth'], data_resampled_smooth.iloc[i]['Longitude_smooth'])
    previous_coord = (data_resampled_smooth.iloc[i-1]['Latitude_smooth'], data_resampled_smooth.iloc[i-1]['Longitude_smooth'])
    if current_coord == previous_coord:
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)

# Save the map
map_filepath = 'walk_pattern_map_segments.html'
m.save(map_filepath)

# Display the map in the notebook
#m


In [ ]:
from itertools import cycle

# Identify the crossover points and their respective segments
crossed_path_indices_set = set(crossed_path_indices)
segments = []
segment = [0]  # Start with the first point
for i in range(1, len(data_resampled_smooth) - 1):
    segment.append(i)
    # If the current point is a crossover point and the next point is not, start a new segment
    if i in crossed_path_indices_set and (i + 1) not in crossed_path_indices_set:
        segments.append(segment)
        segment = [i]  # Start a new segment from the current point
segment.append(len(data_resampled_smooth) - 1)  # Add the last point to the last segment
segments.append(segment)

# Create a new map to visualize the different segments with different colors
m = folium.Map(location=[data_resampled_smooth.iloc[0]['Latitude_smooth'], data_resampled_smooth.iloc[0]['Longitude_smooth']], zoom_start=16)

# Define a color cycle iterator to loop over a set of colors for different segments
colors = cycle(['blue', 'green', 'red', 'purple', 'orange', 'darkblue', 'darkgreen', 'darkred'])

# Plot each segment with a different color
for segment in segments:
    segment_coords = [(data_resampled_smooth.iloc[i]['Latitude_smooth'], data_resampled_smooth.iloc[i]['Longitude_smooth']) for i in segment]
    folium.PolyLine(segment_coords, color=next(colors)).add_to(m)

# Highlight the areas where the person has been stationary for at least 1 minute
for i in range(1, len(data_resampled_smooth)):
    current_coord = (data_resampled_smooth.iloc[i]['Latitude_smooth'], data_resampled_smooth.iloc[i]['Longitude_smooth'])
    previous_coord = (data_resampled_smooth.iloc[i-1]['Latitude_smooth'], data_resampled_smooth.iloc[i-1]['Longitude_smooth'])
    if current_coord == previous_coord:
        folium.CircleMarker(location=current_coord, radius=3, color='yellow', fill=True, fill_color='yellow').add_to(m)

# Save the map to an HTML file
map_filepath_segments_continuous = 'walk_pattern_map_segments_continuous.html'
m.save(map_filepath_segments_continuous)

# Display the map in the notebook
m
